In [4]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-3.5-turbo-1106",
)

alpha_vantage_api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(
        description="The query you will search for.Example query: Stock Market Symbol for Apple Company"
    )


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    
    """
    args_schema: Type[
        StockMarketSymbolSearchToolArgsSchema
    ] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query):
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverviewArgsSchema(BaseModel):
    symbol: str = Field(description="Stock symbol of the company. Example: AAPL, TSLA")


class CompanyOverviewTool(BaseTool):
    name = "CompanyOverview"
    description = """
    Use this to get an overview of the financial of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatement"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()["annualReports"]


class CompanyStockPerformanceTool(BaseTool):
    name = "CompanyStockPerformance"
    description = """
    Use this to get the weekly performance of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverviewArgsSchema] = CompanyOverviewArgsSchema

    def _run(self, symbol):
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}"
        )
        return r.json()


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    handle_parsing_errors=True,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverviewTool(),
        CompanyIncomeStatementTool(),
        CompanyStockPerformanceTool(),
    ],
)

prompt = """
Give me information on Cloudflare's stock, considering its financial, income statements and stock performance.
Help me analyze if it's a potential good investment.
"""

agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'Cloudflare stock market symbol'}`


Cloudflare, Inc. - Stock Quote & Chart Stock Quote & Chart Stock Quote NYSE: NET 80.97 Jan 10, 2024 4:00 PM ET Today's Change -0.66 (-0.81%) Today's Volume 4,138,225 Today's Open 82.55 Previous Close 81.63 Intraday High 83.00 Intraday Low 80.36 52 Week High 86.91 52 Week Low 39.90 Pricing delayed by 20 minutes. Last Updated Jan 10, 2024 4:00 PM ET January 10, 2024 — 06:00 pm EST Written by Zacks Equity Research for Zacks -> The most recent trading session ended with Cloudflare (NET) standing at $80.97, reflecting a -0.81% shift from the... Jan 5, 2024 Fair Value $52.00 Ticker Symbol NET Full Company Profile Financial Performance In 2022, Cloudflare's revenue was $975.24 million, an increase of 48.57% compared to the previous year's $656.43 million. Losses were -$193.38 million, -25.71% less than in 2021. Financial Statements 63.13K followers $76.28 -0.55

{'input': "\nGive me information on Cloudflare's stock, considering its financial and income statements\nand help me analyze if it's a potential good investment.\n",
 'output': "Cloudflare Inc. (NYSE: NET) is a technology company that operates a cloud platform offering a range of network services to companies worldwide. Here's a summary of the financial and income statement data for Cloudflare:\n\nFinancial Overview:\n- Market Capitalization: $27.14 billion\n- Revenue (TTM): $1.21 billion\n- Gross Profit (TTM): $742.63 million\n- EBITDA: -$81.73 million\n- Price to Sales Ratio (TTM): 22.5\n- Price to Book Ratio: 39.2\n- 52-Week High: $86.91\n- 52-Week Low: $39.90\n- 50-Day Moving Average: $75.30\n- 200-Day Moving Average: $65.03\n- Shares Outstanding: 295.9 million\n- Quarterly Revenue Growth (YoY): 32.2%\n\nIncome Statement (2022):\n- Total Revenue: $975.24 million\n- Gross Profit: $742.63 million\n- Operating Income: -$201.20 million\n- Net Income: -$193.38 million\n- EBITDA: -$170.5